<a href="https://colab.research.google.com/github/pranoyk/training-QA-model/blob/main/training_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json
with open(r"train.json", "r") as read_file:
  train = json.load(read_file)

In [3]:
train

[{'context': "module icons { \n yang-version 1.1; \n namespace 'http://anutanetworks.com/icons'; \n prefix ico; \n\n organization \n 'Anuta Networks'; \n \n revision 2021-06-30 { \n description \n 'Report Infra Icons'; \n } \n\n container icons { \n list category { \n key 'name'; \n leaf name { \n type string; \n } \n leaf description { \n type string; \n } \n list icon { \n key 'name'; \n leaf name { \n type string; \n } \n leaf description { \n type string; \n } \n leaf url { \n type string; \n } \n } \n } \n } \n } \n. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/icons:icons/category]",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Retrieve all the available icons?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/icons:icons/category',
      'answer_start': 580}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'Give me all the icons?',
    

In [4]:
import json
with open(r"test.json", "r") as read_file:
  test = json.load(read_file)

In [5]:
test

[{'context': "module alert-group { \n namespace 'http://example.com/alert-group'; \n prefix alert-group; \n  \ncontainer alert-groups { \n container config { \n leaf-list name { \n type string; \n } \n } \n list group-config { \n key 'tenant'; \n leaf tenant { \n type string; \n } \n leaf-list name { \n type string; \n } \n } \n } \n }. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/alert-group:alert-groups]",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Retrieve all the alert groups?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
      'answer_start': 432}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'Can you give me all the alert groups?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
      'answer_start': 432}]}]}]

In [6]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [25]:
model_type = "bert"
model_name = "bert-base-uncased"

In [ ]:
!pip install wandb

In [9]:
args = QuestionAnsweringArgs()


In [26]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    # "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 128,
    "eval_batch_size": 64,
}

In [27]:
!rm -rf outputs

In [28]:
model = QuestionAnsweringModel(
    model_type, model_name, args=train_args
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [29]:
model.train_model(train, eval_data=test)

OutOfMemoryError: ignored